# 課題1

別途支給するテスト用画像(136枚)から、ナンバープレートを検出するプログラムを開発してください。

#### 目標

ナンバープレートを検出する学習モデルの生成

#### 手順

##### 1. 訓練用データを配置する

画像(.jpg)とアノテーション(.txt)のデータセットを訓練用(train)と評価用(val)に分割して本ディレクトリに配置する。


```
datasets
├　images
│　├　train
│　│　└　画像(.jpg)
│　└　val
│　 　└　YOLO形式(.txt)
└　labels
 　├　train
 　│　└　画像(.jpg)
 　└　val
 　 　└　YOLO形式(.txt)
```

##### 2. 訓練する

直下セル実行

重みファイルがyolov5/runs/train/license_plate_detector/weight/に生成される。

In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 100 --data dataset.yml --weights yolov5/yolov5m.pt --name license_plate_detector

# 課題2

検出したナンバープレートにBounding Boxを描画した画像を出力してください。

#### 目標

訓練済みモデルを使用したBounding Boxの推定

元画像にBounding Boxを描画

#### 手順

##### 0. 課題1のコマンドを実行して、モデルを訓練する

##### 1. テストデータを配置するｓ

Bounding Boxを描画したい画像を配置

```
tests
└　input
 　└　画像(.jpg)
```

##### 2. Bounding Boxを描画する

直下セル実行

出力先のディレクトリ

```
tests
└　output
 　└　画像(.jpg)
```

In [ ]:
import math
import torch
import cv2
import glob
import os

# 訓練済みモデル
model = torch.hub.load(
    "yolov5", # モデルを読み込むディレクトリ
    "custom", # モデルの名前
    path="yolov5/runs/train/license_plate_detector/weights/best.pt", # 重みファイルのパス
    source="local" # gitの公開モデルではなく、ローカルのモデルを使用する
)

# 検出しきい値
model.conf = 0.5

input_img_data_path = "tests/input"
output_img_data_path = "tests/output"

# 各画像への処理
# Bounding Boxの計算と描画
for data_path in glob.glob("{0}/*".format(input_img_data_path)):
    file_name = os.path.splitext(os.path.basename(data_path))[0]
    img = cv2.imread(data_path)
    result = model(img)
    for row in result.pandas().xyxy[0].itertuples():
        cv2.rectangle(
            img, 
            (math.floor(row.xmin), math.floor(row.ymin)), # Bounding Boxの左上座標
            (math.floor(row.xmax), math.floor(row.ymax)), # Bounding Boxの右下座標
            color=(255, 255, 0), # 青クリーム色の枠線
            thickness=2 # 枠線の太さ
        )
    cv2.imwrite("{0}/{1}.jpg".format(output_img_data_path, file_name), img)

# 課題3

黄色のナンバープレート(軽自動車)の件数を出力してください。

#### 目標

訓練モデルを使用したBounding Boxの推定

Bouding Boxの半分サイズのフィルターで画像全体を平滑化（ナンバープレートの代表色を作り出すため）

平滑化画像のうち、Bouding Boxの中心のRGBを取得

ナンバープレートの色の候補からRGBに最も近いものを推定


#### 手順

##### 0. 課題1のコマンドを実行して、モデルを訓練する

##### 1. テストデータを配置する

Bounding Boxを描画したい画像を配置

```
tests
└　input
 　└　画像(.jpg)
```

##### 2. 黄色のナンバープレートを数える

直下セル実行

黄色のナンバープレートの数が出力される。



